# Crashes on a Street

In [1]:
import pytz
import folium
from folium.plugins import HeatMap

import pandas as pd
from datetime import datetime, timedelta
from crash_data_analysis import CrashDataAnalysis

In [2]:
cda = CrashDataAnalysis()

In [3]:
start_date = pytz.timezone('America/Denver').localize(datetime(2022,1,1))
end_date = pytz.timezone('America/Denver').localize(datetime(2023,1,1))
most_recent_crash = cda.most_recent_crash_timestamp()

In [4]:
query = f"""
select
sr.gid
, sr.lrsroute
, st_length(sr.geom_denver) / 5280 as length_miles
, c.sbi
, c.fatality
, c.pedestrian_ind
, c.bicycle_ind
, c.geo_lon
, c.geo_lat
, c.reported_date
, c.reported_date at time zone 'America/Denver' as reported_date_local

from street_routes sr
inner join crashes c on st_dwithin(sr.geom_denver, c.geom_denver, 25)

    where sr.gid = 136 -- E 18th Ave
and reported_date at time zone 'America/Denver' > '{start_date.strftime('%Y-%m-%d')}'
and reported_date at time zone 'America/Denver' < '{end_date.strftime('%Y-%m-%d')}'

order by reported_date desc
"""

# where lrsroute ~ '18TH     AV'

street_crashes = pd.read_sql(query, cda.conn)

In [5]:
street_crashes.groupby(street_crashes.reported_date_local.dt.year).agg(
    crashes=('reported_date_local', 'count')
    , fatalities=('fatality', 'sum')
    , sbi=('sbi', 'sum')
    , pedestrian_ind=('pedestrian_ind', 'sum')
    , bicycle_ind=('bicycle_ind', 'sum')
).astype(int)

,crashes,fatalities,sbi,pedestrian_ind,bicycle_ind
reported_date_local,,,,,
2022,80,0,2,2,1


In [6]:
this_map = folium.Map(prefer_canvas=True)
HeatMap(street_crashes[['geo_lat', 'geo_lon']]).add_to(this_map)
this_map.fit_bounds(this_map.get_bounds())
this_map

In [7]:
days_in_data = (end_date - start_date).days

In [8]:
days_between_crashes = days_in_data / len(street_crashes)
days_between_crashes

4.5625

In [9]:
crashes_per_mile_per_week = (len(street_crashes) / street_crashes.iloc[0].length_miles) / (days_in_data/7)
crashes_per_mile_per_week

0.9694417262155868